In [22]:
import cv2, os, random
import pickle, joblib
#https://blog.csdn.net/yinhui_zhang/article/details/83413026
import pandas as pd
from PIL import Image
import numpy as np
from math import floor
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import segmentation_models as sm
import albumentations as A
from collections import Counter

In [23]:
keys = {'成熟度_一溝黃':1,
        '成熟度_二~三溝黃':2,
        '成熟度_四~五溝黃':3,
        '成熟度_催熟後(50-75%黃)':4,
        '成熟度_全黃(75-100%黃)':5,
        '果實受傷':6,
        '其他':7}

data_label = pd.read_excel('data_label.xlsx')
data_label['image_0'] = '01_dataset/papaya/'+data_label['image_0']
data_label['image_1'] = '01_dataset/papaya/'+data_label['image_1']
data_other = [[x,random.choices([x for x in os.listdir('01_dataset/other')])[0]] for x in os.listdir('01_dataset/other')]
data_other = pd.DataFrame(data_other)
data_other.columns = ['image_0', 'image_1']
data_other['image_0'] = '01_dataset/other/'+data_other['image_0']
data_other['image_1'] = '01_dataset/other/'+data_other['image_1']
data_other['class'] = '其他'
data_other['index'] = range(len(data_label), len(data_label)+len(data_other))
data_label = pd.concat([data_label,data_other]).reset_index(drop = True)
data_label['Y'] = data_label['class'] 
data_label = data_label.replace({"Y": keys})
data_label = data_label[data_label['Y']!=6].reset_index(drop = True) #移除果實受傷
training_x, testing_x, _, _  = train_test_split(data_label, 
                                                data_label['Y'].tolist(),
                                                test_size=0.1, random_state=0, 
                                                stratify = data_label['Y'].tolist())
training_x = training_x.reset_index(drop = True)
testing_x = testing_x.reset_index(drop = True)

In [24]:
# 資料修改
testing_x.loc[testing_x['index']==43, 'class'] = '成熟度_催熟後(50-75%黃)' #3
testing_x.loc[testing_x['index']==134, 'class'] = '成熟度_催熟後(50-75%黃)' #3
testing_x.loc[testing_x['index']==487, 'class'] = '成熟度_四~五溝黃' #2
testing_x.loc[testing_x['index']==502, 'class'] = '成熟度_四~五溝黃' #2
testing_x.loc[testing_x['index']==664, 'class'] = '成熟度_四~五溝黃' #2
testing_x.loc[testing_x['index']==828, 'class'] = '成熟度_四~五溝黃' #2
testing_x.loc[testing_x['index']==843, 'class'] = '果實受傷' #X
testing_x.loc[testing_x['index']==880, 'class'] = '成熟度_一溝黃' #0
testing_x.loc[testing_x['index']==1052, 'class'] = '成熟度_一溝黃' #0
testing_x.loc[testing_x['index']==1366, 'class'] = '果實受傷' #全青 #X
testing_x.loc[testing_x['index']==615, 'class'] = '成熟度_全黃(75-100%黃)'
testing_x.loc[testing_x['index']==680, 'class'] = '成熟度_全黃(75-100%黃)'
testing_x.loc[testing_x['index']==915, 'class'] = '成熟度_一溝黃'
testing_x.loc[testing_x['index']==977, 'class'] = '成熟度_一溝黃'
testing_x.loc[testing_x['index']==1045, 'class'] = '成熟度_一溝黃'
testing_x.loc[testing_x['index']==1259, 'class'] = '成熟度_二~三溝黃'
testing_x.loc[testing_x['index']==1309, 'class'] = '成熟度_二~三溝黃'
testing_x.loc[testing_x['index']==1375, 'class'] = '成熟度_二~三溝黃'
testing_x['Y'] = testing_x['class']
testing_x = testing_x.replace({"Y": keys})
testing_x = testing_x[testing_x['Y']!=6].reset_index(drop = True) #移除果實受傷

In [34]:
def image_prepare(input_, img_size = 320):
    bgr_image = cv2.imread(input_)
    # bgr to rgb
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    # padding to square
    max_size = max(rgb_image.shape)
    delta_w = max_size - rgb_image.shape[1]
    delta_h = max_size - rgb_image.shape[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    padding_image = cv2.copyMakeBorder(rgb_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[255, 255, 255])
    # resize
    resize_image = cv2.resize(padding_image, (img_size, img_size), interpolation=cv2.INTER_AREA)
    # rotate
    return [np.rot90(resize_image, k = x) for x in range(4)]

def image_combine(input_1, input_2):
    image_0_rot = image_prepare(input_1)
    image_1_rot = image_prepare(input_2)
    img_0 = sum([[np.concatenate([x, y],axis=1) for x in image_0_rot] for y in image_1_rot], [])
    img_1 = sum([[np.concatenate([x, y],axis=1) for x in image_1_rot] for y in image_0_rot], [])
    return img_0+img_1

def model_loading(file = '04_unet_training/best_model.h5', CLASSES = ['papaya'], img_size = 320):
    os.environ["SM_FRAMEWORK"] = "tf.keras"
    BACKBONE = 'resnet18'
    n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1) 
    activation = 'sigmoid' if n_classes == 1 else 'softmax'
    preprocess_input = sm.get_preprocessing(BACKBONE)
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=(img_size, img_size*2, 3))
    model.load_weights(file) 
    preprocess_input = sm.get_preprocessing(BACKBONE)
    return model, preprocess_input

def detect_papaya(max_index_, contours_, result_):
    x_min = min(contours_[max_index_][:,0][:,0])
    x_max = max(contours_[max_index_][:,0][:,0])
    y_min = min(contours_[max_index_][:,0][:,1])
    y_mix = max(contours_[max_index_][:,0][:,1])
    result_ = result_[y_min:y_mix, x_min:x_max]
    result_ = result_[:,:,:3]    
    if result_.shape[0]<result_.shape[1]:
        return np.rot90(result_, k = 1)
    else:
        return result_

def img_pad(img_):
    if img_.shape[0]<224:
        img_pad_1 = np.zeros((224,img_.shape[1],3))
        img_pad_1[img_pad_1==0] = 255
        w = floor((224-img_.shape[0])/2), (floor((224-img_.shape[0])/2)+img_.shape[0])
        img_pad_1[w[0]:w[1],:img_.shape[1],:img_.shape[2]] = img_
    if img_.shape[0]>224:
        img_pad_1 = cv2.resize(img_, (floor(224*img_.shape[1]/img_.shape[0]), 224), interpolation=cv2.INTER_AREA)
    if img_.shape[0]==224:
        img_pad_1 = img_
        
    if img_pad_1.shape[1]<112:
        img_pad = np.zeros((img_pad_1.shape[0],112,3))
        img_pad[img_pad==0] = 255
        w = floor((112-img_pad_1.shape[1])/2), (floor((112-img_pad_1.shape[1])/2)+img_pad_1.shape[1])
        img_pad[:img_pad_1.shape[0],w[0]:w[1],:img_pad_1.shape[2]] = img_pad_1
    if img_pad_1.shape[1]>112:
        img_pad = cv2.resize(img_pad_1, (112, floor(112*img_pad_1.shape[0]/img_pad_1.shape[1])), interpolation=cv2.INTER_AREA)
    if img_pad_1.shape[1]==112:
        img_pad = img_pad_1
        
    if img_pad.shape[0]<224:
        img_pad_ = np.zeros((224,img_pad.shape[1],3))
        img_pad_[img_pad_==0] = 255
        w = floor((224-img_pad.shape[0])/2), (floor((224-img_pad.shape[0])/2)+img_pad.shape[0])
        img_pad_[w[0]:w[1],:img_pad.shape[1],:img_pad.shape[2]] = img_pad
    else:
        img_pad_ = img_pad
        
    img_pad_ = img_pad_.astype('uint8')
    return img_pad_

def img_combine(data_, add = True):
    image_0_rot = [np.rot90(data_[0], k = x) for x in [0,2]]
    image_1_rot = [np.rot90(data_[1], k = x) for x in [0,2]]
    if add:
        img_0 = sum([[np.concatenate([x, y],axis=1) for x in image_0_rot] for y in image_1_rot], [])
        img_1 = sum([[np.concatenate([x, y],axis=1) for x in image_1_rot] for y in image_0_rot], [])
        img_ = img_0+img_1
        return img_
    else:
        img_0 = sum([[[x,y] for y in image_1_rot] for x in image_0_rot], [])
        img_1 = sum([[[x,y] for y in image_0_rot] for x in image_1_rot], [])
        img_ = img_0+img_1
        return img_

def data_prepare(ind_, dataset, model_, preprocess_input_, single = False):
    img = image_combine(dataset['image_0'][ind_], dataset['image_1'][ind_])
    img_ = preprocess_input_(img[0])
    img_ = np.expand_dims(img_, axis = 0)
    mask = model_.predict(img_).squeeze() 
    mask[mask<0.5] = 0
    mask[mask>=0.5] = 255
    mask = mask.astype('uint8')

    result = img[0].copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    result[mask==0,0] = 255
    result[mask==0,1] = 255
    result[mask==0,2] = 255

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)>=2:
        max_index = list(map(lambda x:len(x), contours))
        max_index_ = max_index.copy()
        max_index_.sort(reverse=True)
        max_index_ = [max_index.index(x) for x in max_index_[:2]]
        crop_img = list(map(lambda x: detect_papaya(x, contours, result), max_index_))
        data = [img_pad(x) for x in crop_img]
        #print(data[0].shape,data[1].shape)
        data_1 = img_combine(data) #兩張圖合併在一起 1*224*224*3
        data_2 = [one_img_pad(x) for x in data]
        #print(data_2[0].shape,data_2[1].shape)
        if single:
            image_0_rot = [np.rot90(data_2[0], k = x) for x in [0,2]]
            image_1_rot = [np.rot90(data_2[1], k = x) for x in [0,2]]
            data = image_0_rot + image_1_rot
            ststus = True
        else:
            data_2_1 = img_combine(data_2, add = False) #multi-input 2*224*224*3
            data_2_2 = [np.concatenate(x, 2) for x in data_2_1] #1*224*224*6
            data = [data_1, data_2_1, data_2_2]
            ststus = True
    else:
        crop_img = None
        data = None
        ststus = False
        
    return mask, result, crop_img, data, ststus

def one_img_pad(img_, center = True):
    img_pad = np.zeros((max(img_.shape),max(img_.shape),3))
    img_pad[img_pad==0] = 255
    if center:
        w = (floor((img_.shape[0]-img_.shape[1])/2)),(floor((img_.shape[0]-img_.shape[1])/2)+img_.shape[1])
        img_pad[:img_.shape[0],w[0]:w[1],:img_.shape[2]] = img_
    else:
        img_pad[:img_.shape[0],:img_.shape[1],:img_.shape[2]] = img_
    img_pad = img_pad.astype('uint8')
    return img_pad

def make_mask(img_):
    mask = (img_ == 255).all(axis=2).astype(int)
    mask = mask^(mask&1==mask)
    return mask
def merge_channel_img(img_, color_ = 255):
    mask = [make_mask(x) for x in img_]
    img_new = [img_[x]*np.stack([mask[x], mask[x], mask[x]], axis=2) for x in range(len(img_))]
    img_new = np.sum(img_new, axis=0)
    mask = np.sum(mask, axis=0)
    mask[mask==0] = 4
    img_new = img_new/np.stack([mask,mask,mask], axis=2)
    mask = (img_new == 0).all(axis=2).astype(int)*color_
    mask = np.stack([mask, mask, mask], axis=2)
    img_new = img_new+mask
    if color_==255:
        img_new = img_new.astype(np.uint8)
    return img_new

def transforms(p=1):
    return A.Compose([
        A.RandomToneCurve(scale=0.3, p=1),
        #A.CLAHE(p=0.5),
        #A.ColorJitter(brightness=1,contrast=0, saturation=0, hue=0),
        #A.RandomBrightnessContrast(),
        A.RandomGamma(p=1),
        A.Blur(blur_limit=3, p=1)
    ], p=p)
def augmentor(image):
    aug = transforms()
    aug_img = aug(image = np.array(image).astype('uint8'))['image']
    #aug_img = preprocess_input(aug_img)
    return aug_img

In [26]:
model, preprocess_input = model_loading()

### classification_dataset_chanel3

In [8]:
from collections import Counter

training_dict = dict()
for ind in range(len(training_x)):
    print(ind, len(training_x))
    _, _, _, img, ststus = data_prepare(ind, training_x, model, preprocess_input, single = True)
    if (ststus == True):
        img = [merge_channel_img(img)]+[merge_channel_img([augmentor(x) for x in img]) for x in range(7)]
        tmp = [{x:{"Y":training_x['Y'][ind], 'image': img[int(x.split("_")[1])]}} \
               for x in [str(training_x['index'][ind]).rjust(4, '0')+"_"+str(x) for x in range(8)]]
        tmp = dict(map(dict.popitem, tmp))
        training_dict.update(tmp)
        del tmp
training_dict = list(training_dict.items())
random.shuffle(training_dict)
training_dict = dict(training_dict)

testing_dict = dict()
for ind in range(len(testing_x)):
    print(ind, len(testing_x))
    _, _, _, img, ststus = data_prepare(ind, testing_x, model, preprocess_input, single = True)
    if (ststus == True):
        img = [merge_channel_img(img)]+[merge_channel_img([augmentor(x) for x in img]) for x in range(7)]
        tmp = [{x:{"Y":testing_x['Y'][ind], 'image': img[int(x.split("_")[1])]}} \
               for x in [str(testing_x['index'][ind]).rjust(4, '0')+"_"+str(x) for x in range(8)]]
        tmp = dict(map(dict.popitem, tmp))
        testing_dict.update(tmp)
        del tmp
testing_dict = list(testing_dict.items())
random.shuffle(testing_dict)
testing_dict = dict(testing_dict)

training_key = list(training_dict.keys())
testing_key = list(testing_dict.keys())
count = Counter([training_dict[x]['Y'] for x in training_key])
all_dict = {"training_Y":np.array(pd.get_dummies([training_dict[x]['Y'] for x in training_key])),
            "training_X":training_dict,
            "testing_Y":np.array(pd.get_dummies([testing_dict[x]['Y'] for x in testing_key])),
            "testing_X":testing_dict,
            "training_count":count}
with open('06_classification_image/classification_dataset_chanel3.pkl', 'wb') as f:
    pickle.dump(all_dict, f)

0 1961


2022-11-07 16:14:11.148942: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


1 1961
2 1961
3 1961
4 1961
5 1961
6 1961
7 1961
8 1961
9 1961
10 1961
11 1961
12 1961
13 1961
14 1961
15 1961
16 1961
17 1961
18 1961
19 1961
20 1961
21 1961
22 1961
23 1961
24 1961
25 1961
26 1961
27 1961
28 1961
29 1961
30 1961
31 1961
32 1961
33 1961
34 1961
35 1961
36 1961
37 1961
38 1961
39 1961
40 1961
41 1961
42 1961
43 1961
44 1961
45 1961
46 1961
47 1961
48 1961
49 1961
50 1961
51 1961
52 1961
53 1961
54 1961
55 1961
56 1961
57 1961
58 1961
59 1961
60 1961
61 1961
62 1961
63 1961
64 1961
65 1961
66 1961
67 1961
68 1961
69 1961
70 1961
71 1961
72 1961
73 1961
74 1961
75 1961
76 1961
77 1961
78 1961
79 1961
80 1961
81 1961
82 1961
83 1961
84 1961
85 1961
86 1961
87 1961
88 1961
89 1961
90 1961
91 1961
92 1961
93 1961
94 1961
95 1961
96 1961
97 1961
98 1961
99 1961
100 1961
101 1961
102 1961
103 1961
104 1961
105 1961
106 1961
107 1961
108 1961
109 1961
110 1961
111 1961
112 1961
113 1961
114 1961
115 1961
116 1961
117 1961
118 1961
119 1961
120 1961
121 1961
122 1961
123 1961
1

924 1961
925 1961
926 1961
927 1961
928 1961
929 1961
930 1961
931 1961
932 1961
933 1961
934 1961
935 1961
936 1961
937 1961
938 1961
939 1961
940 1961
941 1961
942 1961
943 1961
944 1961
945 1961
946 1961
947 1961
948 1961
949 1961
950 1961
951 1961
952 1961
953 1961
954 1961
955 1961
956 1961
957 1961
958 1961
959 1961
960 1961
961 1961
962 1961
963 1961
964 1961
965 1961
966 1961
967 1961
968 1961
969 1961
970 1961
971 1961
972 1961
973 1961
974 1961
975 1961
976 1961
977 1961
978 1961
979 1961
980 1961
981 1961
982 1961
983 1961
984 1961
985 1961
986 1961
987 1961
988 1961
989 1961
990 1961
991 1961
992 1961
993 1961
994 1961
995 1961
996 1961
997 1961
998 1961
999 1961
1000 1961
1001 1961
1002 1961
1003 1961
1004 1961
1005 1961
1006 1961
1007 1961
1008 1961
1009 1961
1010 1961
1011 1961
1012 1961
1013 1961
1014 1961
1015 1961
1016 1961
1017 1961
1018 1961
1019 1961
1020 1961
1021 1961
1022 1961
1023 1961
1024 1961
1025 1961
1026 1961
1027 1961
1028 1961
1029 1961
1030 1961
1031 1

1751 1961
1752 1961
1753 1961
1754 1961
1755 1961
1756 1961
1757 1961
1758 1961
1759 1961
1760 1961
1761 1961
1762 1961
1763 1961
1764 1961
1765 1961
1766 1961
1767 1961
1768 1961
1769 1961
1770 1961
1771 1961
1772 1961
1773 1961
1774 1961
1775 1961
1776 1961
1777 1961
1778 1961
1779 1961
1780 1961
1781 1961
1782 1961
1783 1961
1784 1961
1785 1961
1786 1961
1787 1961
1788 1961
1789 1961
1790 1961
1791 1961
1792 1961
1793 1961
1794 1961
1795 1961
1796 1961
1797 1961
1798 1961
1799 1961
1800 1961
1801 1961
1802 1961
1803 1961
1804 1961
1805 1961
1806 1961
1807 1961
1808 1961
1809 1961
1810 1961
1811 1961
1812 1961
1813 1961
1814 1961
1815 1961
1816 1961
1817 1961
1818 1961
1819 1961
1820 1961
1821 1961
1822 1961
1823 1961
1824 1961
1825 1961
1826 1961
1827 1961
1828 1961
1829 1961
1830 1961
1831 1961
1832 1961
1833 1961
1834 1961
1835 1961
1836 1961
1837 1961
1838 1961
1839 1961
1840 1961
1841 1961
1842 1961
1843 1961
1844 1961
1845 1961
1846 1961
1847 1961
1848 1961
1849 1961
1850 1961


### classification_dataset_chanel12

In [5]:
from collections import Counter

training_dict = dict()
for ind in range(len(training_x)):
    _, _, _, img, ststus = data_prepare(ind, training_x, model, preprocess_input)
    if (ststus == True):
        img_list = img[1][0]+img[1][1]
        tmp = [{x:{"Y":training_x['Y'][ind], 'image': random.sample(img_list, len(img_list))}} \
               for x in [str(training_x['index'][ind]).rjust(4, '0')+"_"+str(x) for x in range(8)]]
        tmp = dict(map(dict.popitem, tmp))
        training_dict.update(tmp)
        del tmp
training_dict = list(training_dict.items())
random.shuffle(training_dict)
training_dict = dict(training_dict)

testing_dict = dict()
for ind in range(len(testing_x)):
    _, _, _, img, ststus = data_prepare(ind, testing_x, model, preprocess_input)
    if (ststus == True):
        img_list = img[1][0]+img[1][1]
        tmp = [{x:{"Y":testing_x['Y'][ind], 'image': random.sample(img_list, len(img_list))}} \
               for x in [str(testing_x['index'][ind]).rjust(4, '0')+"_"+str(x) for x in range(8)]]
        tmp = dict(map(dict.popitem, tmp))
        testing_dict.update(tmp)
        del tmp
testing_dict = list(testing_dict.items())
random.shuffle(testing_dict)
testing_dict = dict(testing_dict)

training_key = list(training_dict.keys())
testing_key = list(testing_dict.keys())
count = Counter([training_dict[x]['Y'] for x in training_key])
all_dict = {"training_Y":np.array(pd.get_dummies([training_dict[x]['Y'] for x in training_key])),
            "training_X":training_dict,
            "testing_Y":np.array(pd.get_dummies([testing_dict[x]['Y'] for x in testing_key])),
            "testing_X":testing_dict,
            "training_count":count}
with open('06_classification_image/classification_dataset_chanel12.pkl', 'wb') as f:
    pickle.dump(all_dict, f)

2022-11-01 20:18:55.641343: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


### data prepare

In [5]:
error = []
merge_image = dict()
multi_image = dict()
chanel_image = dict()
for ind in range(len(training_x)):
    mask, result, crop_img, img, ststus = data_prepare(ind, training_x, model, preprocess_input)
    if (ststus == False):
        if(training_x.iloc[ind]['class']!='其他'):
            print("other", ind)
            error.append(ind)
    else:
        if (len(np.unique([x.shape[0] for x in img[0]]))==1)&(img[0][0].shape[0]==224)& \
           (len(np.unique([x.shape[1] for x in img[0]]))==1)&(img[0][0].shape[1]==224):
            for j in range(8):
                n_ = str(training_x.iloc[ind]['index']).rjust(4, '0')+"_"+str(j)+".jpg"
                Image.fromarray(img[0][j]).save('06_classification_image/train/' + n_, quality=100)
                merge_image[n_] = img[0][j]
                multi_image[n_] = img[1][j]
                chanel_image[n_] = img[2][j]
        else:
            print("error", ind)
            error.append(ind)
print(error)
with open('06_classification_image/classification_dataset_training.pkl', 'wb') as f:
    pickle.dump({'merge_image':merge_image, 'multi_image':multi_image, 'chanel_image':chanel_image}, f)

error = []
merge_image = dict()
multi_image = dict()
chanel_image = dict()
for ind in range(len(testing_x)):
    mask, result, crop_img, img, ststus = data_prepare(ind, testing_x, model, preprocess_input)
    if (ststus == False):
        if(testing_x.iloc[ind]['class']!='其他'):
            print("other", ind)
            error.append(ind)
    else:
        if (len(np.unique([x.shape[0] for x in img[0]]))==1)&(img[0][0].shape[0]==224)& \
           (len(np.unique([x.shape[1] for x in img[0]]))==1)&(img[0][0].shape[1]==224):
            for j in range(8):
                n_ = str(testing_x.iloc[ind]['index']).rjust(4, '0')+"_"+str(j)+".jpg"
                Image.fromarray(img[0][j]).save(
                    '06_classification_image/test/' + n_, 
                    quality=100)
                merge_image[n_] = img[0][j]
                multi_image[n_] = img[1][j]
                chanel_image[n_] = img[2][j]
        else:
            print("error", ind)
            error.append(ind)
with open('06_classification_image/classification_dataset_testing.pkl', 'wb') as f:
    pickle.dump({'merge_image':merge_image, 'multi_image':multi_image, 'chanel_image':chanel_image}, f)
print(error)

from collections import Counter

with open('06_classification_image/classification_dataset_training.pkl', 'rb') as f:
    training_dataset = pickle.load(f)
with open('06_classification_image/classification_dataset_testing.pkl', 'rb') as f:
    testing_dataset = pickle.load(f)
    
training_key = [x for x in os.listdir('06_classification_image/train/') if '.jpg' in x]
testing_key = [x for x in os.listdir('06_classification_image/test/') if '.jpg' in x]
count = Counter([training_x[training_x['index']==int(x.split("_")[0])]['Y'].tolist()[0] for x in training_key])
all_dict = {"training_Y":np.array(pd.get_dummies([training_x[training_x['index']==int(x.split("_")[0])]['Y'].tolist()[0] for x in training_key])),
            "testing_Y":np.array(pd.get_dummies([testing_x[testing_x['index']==int(x.split("_")[0])]['Y'].tolist()[0] for x in testing_key])),
            "training_count":count}

2022-11-01 12:26:09.108855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


[]


### classification_dataset_merge

In [8]:
training_dict = [{x:{'Y':training_x[training_x['index']==int(x.split("_")[0])]['Y'].tolist()[0], 
                     'image':training_dataset['merge_image'][x]}} \
                     for x in training_key]
training_dict = dict(map(dict.popitem, training_dict))
testing_dict = [{x:{'Y':testing_x[testing_x['index']==int(x.split("_")[0])]['Y'].tolist()[0],
                    'image':testing_dataset['merge_image'][x]}} \
                 for x in testing_key]
testing_dict = dict(map(dict.popitem, testing_dict))
all_dict["training_X"] = training_dict
all_dict["testing_X"] = testing_dict
with open('06_classification_image/classification_dataset_merge.pkl', 'wb') as f:
    pickle.dump(all_dict, f)

### classification_dataset_ mutil

In [9]:
training_dict = [{x:{'Y':training_x[training_x['index']==int(x.split("_")[0])]['Y'].tolist()[0], 
                     'image':training_dataset['multi_image'][x]}} \
                     for x in training_key]
training_dict = dict(map(dict.popitem, training_dict))
testing_dict = [{x:{'Y':testing_x[testing_x['index']==int(x.split("_")[0])]['Y'].tolist()[0],
                    'image':testing_dataset['multi_image'][x]}} \
                 for x in testing_key]
testing_dict = dict(map(dict.popitem, testing_dict))
all_dict["training_X"] = training_dict
all_dict["testing_X"] = testing_dict
with open('06_classification_image/classification_dataset_multi.pkl', 'wb') as f:
    pickle.dump(all_dict, f)

### classification_dataset_chanel6

In [10]:
training_dict = [{x:{'Y':training_x[training_x['index']==int(x.split("_")[0])]['Y'].tolist()[0], 
                     'image':training_dataset['chanel_image'][x]}} \
                     for x in training_key]
training_dict = dict(map(dict.popitem, training_dict))
testing_dict = [{x:{'Y':testing_x[testing_x['index']==int(x.split("_")[0])]['Y'].tolist()[0],
                    'image':testing_dataset['chanel_image'][x]}} \
                 for x in testing_key]
testing_dict = dict(map(dict.popitem, testing_dict))
all_dict["training_X"] = training_dict
all_dict["testing_X"] = testing_dict
with open('06_classification_image/classification_dataset_chanel6.pkl', 'wb') as f:
    pickle.dump(all_dict, f)